In [1]:
"""
input: time of post (AM or PM)
output: aqi
"""

# import libraries
import pandas as pd
import requests
import json
from datetime import date

# get today's date
day = date.today().day
month = date.today().month
year = date.today().year

# get post time
post_time = input("Enter the time you want to post the bot: 8 or 15 ")
post_time_2 = int(post_time) + 1
        
# pull data from AirNow
r = requests.get('https://www.airnowapi.org/aq/data/?startDate={year}-{month}-{day}T{time1}&endDate={year}-{month}-{day}T{time2}&parameters=PM25&BBOX=-84.500651,30.275370,-84.080424,30.684863&dataType=B&format=application/json&verbose=1&monitorType=2&includerawconcentrations=0&API_KEY=2BB44069-F9EF-4CA7-8B67-C9832B168B60'.format(day = day, month = month, year = year, time1 = post_time, time2 = post_time_2)).json()

# store in dataframe
df = pd.DataFrame.from_dict(r)

# clean data
df['date'] = df['UTC'].str.split('T').str[0]
df['time'] = df['UTC'].str.split('T').str[1]

df = df.drop(['UTC'], axis=1)

df['Category'] = df['Category'].astype(str)

# create dataframe
conditions = pd.DataFrame({
    'category': ['1', '2', '3', '4', '5', '6'],
    'status': ['good', 'moderate', 'unhealthy for sensitive groups', 'unhealthy', 'very unhealthy', 'hazardous'],
    'color': ['green', 'yellow', 'orange', 'red', 'purple', 'maroon'],
    'message': ['In the Tallahassee area, the air quality is "good". Enjoy your outdoor activities!', 
            'In the Tallahassee area, the air quality is "moderate". If you are sensitive to air pollution, limit your time outdoors or wear a mask.',
            'In the Tallahassee area, the air quality is "unhealthy for sensitive groups',
            'In the Tallahassee area, the air quality is "unhealthy".',
            'In the Tallahassee area, the air quality is "very unhealthy',
            'In the Tallahassee area, the air quality is "hazardous.']
})

# add conditions to original dataframe
def add_conditions(dataframe):
    for i in range(len(dataframe)):
        dataframe.loc[i, 'status'] = conditions[conditions['category'] == df.loc[i, 'Category']]['status'].loc[0]
        dataframe.loc[i, 'color'] = conditions[conditions['category'] == df.loc[i, 'Category']]['color'].loc[0]
        dataframe.loc[i, 'message'] = conditions[conditions['category'] == df.loc[i, 'Category']]['message'].loc[0]
    return dataframe

add_conditions(df)

# get formatted time
def format_time(post_time):
    if post_time > 12:
        time = post_time - 12
        hour = str(time) + 'PM'
    elif post_time:
        hour = str(post_time) + 'AM'
    return hour

hour = format_time(int(post_time))

# print status check
print("the AQI for {day} {hour} is {AQI}, category {level}".format(day=date.today(), hour = hour, AQI = df.iloc[0]['AQI'], level = df.iloc[0]['Category']))

# print status check
print("adding AQI reading to the csv")

# store data in csv
with open('/Users/shelbygreen/Repositories/aq-bot/data/log.csv', 'a') as f:
    df.to_csv(f, header=False)

Enter the time you want to post the bot: 8 or 15 21
the AQI for 2021-12-12 9PM is 34, category 1
adding AQI reading to the csv


## Using Pillow for Image Processing

In [4]:
# import libraries
from PIL import Image, ImageDraw, ImageFont
from datetime import date

# set name of file
fileName = conditions[conditions['category'] == df.iloc[0]['Category']].iloc[0]['color']

# generate saying
saying = "As of {month}/{day}/{year}, {time}M:".format(month=month, day=day, year=year, time=post_time)

# store picture / create image object
pic = Image.open('/Users/shelbygreen/Repositories/aq-bot/templates/{color}.png'.format(color=fileName))

# add saying to the picture
# create draw object from image object
draw = ImageDraw.Draw(pic)

# add font and text size
font = ImageFont.truetype("Library/Fonts/GlacialIndifference-Regular.otf", 70)

# draw on image
draw.text((50, 350), '{saying}'.format(saying=saying), fill='#63625E', font=font)

# add AQI to the picture
# add font and text size
font = ImageFont.truetype("Library/Fonts/GlacialIndifference-Bold.otf", 150)

# draw on image
draw.text((90, 470), '{value}'.format(value=df.iloc[0]['AQI']), fill='#000000', font=font)

pic.save('/Users/shelbygreen/Repositories/aq-bot/finished/{month}-{day}-{year}-{post_time}.png'.format(month=month, day=day, year=year, post_time=post_time))

## Using Twippy to post to Twitter

In [7]:
# !pip install schedule
# tweppy

df[(df['date'] == '2021-12-12') & (df['time'] == '21:00')]['message'].loc[0]

'In the Tallahassee area, the air quality is "good". Enjoy your outdoor activities!'

In [3]:
# import libraries
import pandas as pd
import tweepy 
import io
import time
import requests
import os
import yaml
import schedule
from datetime import date

# import csv
df = pd.read_csv('/Users/shelbygreen/Repositories/aq-bot/data/log.csv')

def job():
    print("check1")
    
    # get tweet wording
    tweet_date = '{year}-{month}-{day}'.format(year=year, month=month, day=day)
    tweet_hour = '{time}'.format(time=time)
    tweet_time = '{time}:00'.format(time=post_time)
    tweet_msg = df[(df['date'] == tweet_date) & (df['time'] == tweet_time)]['message'].loc[0]

    # read YAML file
    yaml_file = open("/Users/shelbygreen/Repositories/aq-bot/keys.yaml")
    parsed_yaml_file = yaml.load(yaml_file)
    
    print("check2")

    # twitter authorization function
    def twitter_api():
      auth = tweepy.OAuthHandler(parsed_yaml_file["consumer_key"], parsed_yaml_file["consumer_secret"])
      auth.set_access_token(parsed_yaml_file["access_token"], parsed_yaml_file["access_token_secret"])
      api = tweepy.API(auth)
      return api

    # post tweet function
    def post_tweet():
      api = twitter_api()

      for index, row in df.iterrows():
            if int(time) <= 12:
                # make morning msg
                full_tweet = 'Good morning ☀️' + tweet_msg
                # make tweet
                api.update_with_media('{tweet_date}-{time}.png'.format(tweet_date=tweet_date, time=time), status=tweet_msg)
            else:
                # make afternoon msg
                full_tweet = 'Good afternoon 🍂' + tweet_msg
                # make tweet
                api.update_with_media('{tweet_date}-{time}.png'.format(tweet_date=tweet_date, time=time), status=tweet_msg)

    print("check3")
    #post tweet
    post_tweet()


# run everyday at 8am and 3pm
schedule.every().day.at("07:59").do(job)

schedule.every().day.at("14:59").do(job)

schedule.every().day.at("21:16").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

check1


KeyError: 'date'

## Using Selenium to access Canva -- Didn't Work!

In [1]:
!pip install selenium

# and download driver and store in working directory

In [12]:
from selenium import webdriver # selenium
from time import sleep

In [77]:
# define canva log-in credentials
userName = 'greenshelby@gmail.com'
passWord = '#Sgcaca21!'

In [92]:
# define browser
# this will open Google Chrome to a page called "data:,"
browser = webdriver.Chrome('./chromedriver')

<ipython-input-92-3f0371f94d1f>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('./chromedriver')


In [93]:
# the browser will navigate to this website
browser.get("http://www.canva.com/login")

In [95]:
# find email box and enter email
browser.find_element_by_xpath("//*[(@id = '__a11yId21')]").send_keys(userName)

<ipython-input-95-ddc8534fb8a1>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath("//*[(@id = '__a11yId21')]").send_keys(userName)


In [97]:
# find password box and enter password
browser.find_element_by_xpath("//*[(@id = '__a11yId24')]").send_keys(passWord)

<ipython-input-97-1619a2c6f60b>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath("//*[(@id = '__a11yId24')]").send_keys(passWord)


In [98]:
# find login box and click it
browser.find_element_by_css_selector(".ogth8A ._38oWvQ").click()

<ipython-input-98-540c91a51a83>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_xpath("//*[contains(concat( " ", @class, " " ), concat( " ", 'ogth8A', " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", '_38oWvQ', " " ))]").click()


InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //*[contains(concat( , @class,  ), concat( , 'ogth8A',  ))]//*[contains(concat( , @class,  ), concat( , '_38oWvQ',  ))] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//*[contains(concat( , @class,  ), concat( , 'ogth8A',  ))]//*[contains(concat( , @class,  ), concat( , '_38oWvQ',  ))]' is not a valid XPath expression.
  (Session info: chrome=96.0.4664.93)
Stacktrace:
0   chromedriver                        0x0000000103201269 __gxx_personality_v0 + 582729
1   chromedriver                        0x000000010318cc33 __gxx_personality_v0 + 106003
2   chromedriver                        0x0000000102d49e28 chromedriver + 171560
3   chromedriver                        0x0000000102d4cca1 chromedriver + 183457
4   chromedriver                        0x0000000102d4cac1 chromedriver + 182977
5   chromedriver                        0x00000

In [99]:
# after 5 seconds, the browser will close
sleep(5)
browser.close()

## Creating YAML file

In [110]:
import yaml 

#replace 'AAAA' with the appropriate key
dict_file = {'consumer_key': 'AAAA', # consumer key > API
             'consumer_secret': 'AAAA', # consumer key > secret
             'access_token': 'AAAA', # authentican tokens > access token
             'access_token_secret': 'AAAA'} # authentican tokens > access token secret

with open('./keys.yaml', 'w') as file:
    documents = yaml.dump(dict_file, file)